In [ ]:
import pandas as pd

b = pd.read_csv('./부산/')
u = 
gy 
j
gw

=========================

In [7]:
import requests
import json

def translate(query):
    url = "https://asia-northeast3-skilled-chalice-402604.cloudfunctions.net/translate"

    headers = {'Content-Type' : 'application/json'}
    payload = json.dumps({'queries' : [query]})
    translations = requests.post(
        url = url,
        data = payload,
        headers = headers
    )
    result = translations.json()
    return result['translations'][0]

In [9]:
pinecone_api_key = getpass()

In [10]:
from pinecone import Pinecone

pc = Pinecone(api_key = pinecone_api_key)

In [11]:
index = pc.Index('books')

In [12]:
index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.18718,
 'namespaces': {'': {'vector_count': 18718}},
 'total_vector_count': 18718}

In [16]:
from getpass import getpass

openai_api_key = getpass("OPENAI_TOKEN")

In [17]:
from openai import OpenAI

openai_client = OpenAI(api_key=openai_api_key)

In [18]:
def get_embedding(text_list):
    response = openai_client.embeddings.create(
        input=text_list,
        model = 'text-embedding-3-small',
        dimensions=512
    )
    return [x.embedding for x in response.data]

In [56]:
def recommend(query):
    eng_query = translate(query)
    query_embedding = get_embedding([eng_query])[0]
    results = index.query(
        vector = query_embedding,
        filter={
            'publisher' : {'$in' : ['북박스']}   
        },
        top_k = 2,
        include_metadata = True
    )
    return results['matches']

In [57]:
query = '맛있는 음식 이야기'

In [59]:
recommend(query)

[{'id': '295010',
  'metadata': {'authors': '최은영',
               'img_url': 'https://image.yes24.com/goods/295010/L',
               'published_at': '2002-08',
               'publisher': '북박스',
               'rating': 9.0,
               'review_cnt': 1.0,
               'summary': "천리안 동호회 '천일야화'의 비공개 사이트에 연재된 작품으로 파격적인 내용과 독특한 소재로 "
                          '인해 로맨스 애호가들에게 비상한 관심을 끌었던 작품이다. 개성 강한 인물들의 예측 불허한 삶을 '
                          '그리는 이 특별한 소설은 곳곳에 허를 찌르는 대사를 폭탄처럼 숨겨놓았다가 적시에 한번씩 '
                          '터뜨림으로써 독자들의 가슴을 단번에 사로잡는 작품이다. 또한 거친 폭풍 같은 남자를 사랑하는 '
                          '지고지순한 여인의 따뜻하면서도 꿋꿋한 행보를 강한 반전으로 풀어내 사랑이라는 그릇안에 세상을 '
                          '폭넓게 껴안고 있다. 결국 사랑하는 사람들이 서로 합쳐져 하나의 가정을 이룬다는 의미의 '
                          '플러스는 가정의 소중함을 끝까지 완성시켜 독자들에게 깊은 여운을 남길 것으로 믿는다.',
               'title': '플러스 1',
               'translation': 'It was serialized on the private site of the '
                              "clairvoyant club 'Thousand and One Nights

In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [51]:
url = 'https://korean.visitkorea.or.kr/call'
params = {
    'cmd': 'TOUR_CONTENT_LIST_VIEW',
    'month': 'All',
    'areaCode': '6',
    'sigunguCode': 'All',
    'tagId': 'All',
    'sortkind': '1',
    'locationx': '0',
    'locationy': '0',
    'page': '1',
    'cnt': '100',
    'typeList': 'Tour',
    'stampId': '1589345b-b030-11ea-b8bd-020027310001'
}

all_data = []  # 모든 페이지의 데이터를 저장할 리스트

while True:
    response = requests.get(url, params=params)
    data = response.json()

    # 받은 데이터에서 필요한 정보 추출
    result_list = data.get('body', {}).get('result', [])
    for item in result_list:
        img_path = item.get('imgPath', '')
        img_url = f'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id={img_path}'
        
        extracted_data = {
            '관광지명': item.get('TITLE', ''),
            'catch': item.get('catchPhrase', ''),
            'addr1': item.get('addr1', ''),
            'img': img_url,
            'cotid' : item.get('cotId', ''),
            'areaCode' : item.get('areaCode', ''),
            'cat1' : item.get('cat1', ''),
            'cat2' : item.get('cat2', '')
        }
        
        cotid = extracted_data['cotid']
        cat1 = extracted_data['cat1']
        cat2 = extracted_data['cat2']
        areaCode = extracted_data['areaCode']
        
        new_url = f'https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid={cotid}&big_category={cat1}&mid_category={cat2}&big_area={areaCode}'
        response = requests.get(new_url)
        html = response.text
        soup = BeautifulSoup(html,'html.parser')
        description = soup.find('div',class_='blind').text
        description = description.replace('\n','').replace('\t','')
        extracted_data['description'] = description

        all_data.append(extracted_data)
    
    # 다음 페이지로 이동
    # if params['page'] in range(0,1000,100):
    #     print('time sleep')
    #     time.sleep(200)
    # else:
    
    params['page'] = str(int(params['page']) + 1)
    print(params['page'])
    

    # 만약 더 이상 페이지가 없으면 반복문 종료
    if not result_list or len(all_data) == 1351:
        break

2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [56]:
import pandas as pd
df = pd.DataFrame(all_data)

In [57]:
df

,관광지명,catch,addr1,img,cotid,areaCode,cat1,cat2,description
0,라마다 앙코르 바이윈덤 부산해운대,해운대의 해변과 도시를 모두 찾는 고객에게 편안한 여행을 선사하는 곳,부산광역시 해운대구 구남로 9,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,9be6cf0e-3a4d-439b-97b2-a0c07ca0b3a1,6,B02,B0201,라마다 앙코르 바이윈덤 부산해운대는 세계 최다 호텔을 보유한 윈덤(Wyndham) ...
1,토북베이커리,빠르지만 재치있고 느리지만 정확하게,부산광역시 강서구 낙동남로 526-1,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,4c832d47-15b0-41e7-9108-e9d4f4bc8f94,6,A05,A0502,느리지만 정직한 빵을 만드는 김덕규베이커리에서 새로운 공간으로 부산 강서구에 한옥이...
2,무목,한식 퓨전 디저트 전문점,부산광역시 금정구 체육공원로 689-22 (두구동),https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,834de389-ba5f-4f91-a2b3-17e2c35ff032,6,A05,A0502,"무목은 한식 퓨전 디저트 카페다. ‘목적 없이 흘러들어온 쉼의 터’라는 부제로, 다..."
3,부산진 별빛산책길,빛과 함께 걷는 부산 시민공원 야간 명소거리,부산광역시 부산진구 범전동,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,b94cbf61-90d5-4f3d-b62e-a4303e397e35,6,A02,A0203,부산진 별빛산책길은 고보조명과 빛 조형물을 이용하여 약 400m의 4개 구역 별 야...
4,브라운도트호텔 서면범천점,서면 번화가 인근의 편안한 호텔,부산광역시 부산진구 황령대로7번길 20-11 (범천동),https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,05d32c9a-2aeb-493a-a6f5-f26070529c42,6,B02,B0201,부산의 상업 중심지인 서면 번화가에서 도보 8분 거리에 위치한 브라운도트호텔 서면범...
...,...,...,...,...,...,...,...,...,...
1346,고스락,부산기장 가족단위 조용한 방갈로식당,부산광역시 기장군 장안읍 해맞이로 286,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,5fe09751-7bbc-4a98-9fda-50bca0a01d56,6,A05,A0502,부산 기장군 임랑해수욕장 해변에 위치한 고스락은 시원한 바닷바람과 먹는 신선한 요리...
1347,개화,다양한 중국요리를 선보이다,부산광역시 중구 광복로 21,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,de438b3e-fe80-4724-b28b-2d1cbf6061b0,6,A05,A0502,개화는 부산 중구 부평동 1가에 위치한 중국요리 전문점으로 60년 넘는 세월 동안 ...
1348,좋은쌀로밥짓고,깔끔하고 푸짐한 영양돌솥정식을 맛볼 수 있는 곳,부산광역시 중구 광복중앙로24번길 9,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,83a7ffc7-cf90-41d5-90e1-659d5c01d4e7,6,A05,A0502,좋은 쌀로 밥 짓고는 부산 중구에 위치한 한정식 전문점이다.영양 돌솥 정식 단일 메...
1349,부산 송도해상케이블카,하늘과 바다를 대표하는 비행기와 크루즈의 만남!,부산광역시 서구 송도해변로 171,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,983e127f-9fe4-467b-9f41-dd6926fbe9e4,6,A02,A0203,송도해수욕장에 위치한 송도해상케이블카는 우리나라 제1호 공설해수욕장인 송도해수욕장의...


In [58]:
b = pd.read_csv('./data/busan.csv')

In [71]:
answer = []
names = [i.replace(' ','') for i in df['관광지명'].tolist()]
for i in b['관광지명']:
    if i in names:
        answer.append(i)
    else:
        print(i)

print(len(answer))

학리항
만덕고개누리길전망데크
부산충렬사
재한유엔기념공원
영도하리선착장
안적사
요산문학관
금강사
부산차이나타운특구
마하사
국청사
동백섬
죽성성당
친환경스카이웨이전망대
구상반려암
월전포구
기장해녀촌
성암사
금정산
선암사
델타루
오륙도
법륜사
보광사
홍법사
죽성항
소림사
연등사
내원정사
부산이중섭거리
대각사
청량사
봉래산
묘심사
태종대
운수사
장안사
신선대
영주암
월명사
장산
부곡상가시장
척판암
부산라벤더팜
수산과학관
허브랑야생화
부산범어사
미륵사
부산민주공원
금강암
부산롯데월드
벡스코
금수사
금용암
해동용궁사
부평시장
이기대
부산올림픽공원
문화공감수정
두도
감천사
178


In [67]:
a = '부산 송도 해수욕장'

In [70]:
print(a.replace(' ',''))

부산송도해수욕장


In [45]:
for data in all_data:
    cotid = data['cotid']
    cat1 = data['cat1']
    cat2 = data['cat2']
    areaCode = data['areaCode']
    
    new_url = f'https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid={cotid}&big_category={cat1}&mid_category={cat2}&big_area={areaCode}'
    response = requests.get(new_url)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    description = soup.find('div',class_='blind').text
    description = description.replace('\n','').replace('\t','')
    print(description)
    break    

라마다 앙코르 바이윈덤 부산해운대는 세계 최다 호텔을 보유한 윈덤(Wyndham) 그룹의 글로벌 브랜드 호텔로 해운대역(부산 2호선)에서 바로 앞에 위치해 해운대의 해변과 도시를 모두 찾는 고객에게 가장 편안한 여행의 경험을 선사한다.해운대 해수욕장까지 도보 500m로 아름다운 바다를 가깝게 만날 수 있으며 부산의 맛을 느낄 수 있는 다양한 먹거리도 쉽게 찾을 수 있다. 호텔에서 10분 거리에 위치한 벡스코는 대중교통으로도 이동할 수 있어 비즈니스로 해운대를 방문하시는 고객에게 교통의 편의까지 더하여 최고의 입지를 자랑한다.


In [14]:
all_data[0]

{'관광지명': '라마다 앙코르 바이윈덤 부산해운대',
 'catch': '해운대의 해변과 도시를 모두 찾는 고객에게 편안한 여행을 선사하는 곳',
 'addr1': '부산광역시 해운대구 구남로 9',
 'img': 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=18c244bc-dc04-46d3-8e81-cfe21706935c',
 'cotid': '9be6cf0e-3a4d-439b-97b2-a0c07ca0b3a1',
 'areaCode': '6',
 'cat1': 'B02',
 'cat2': 'B0201'}

In [17]:
html = response.text

In [19]:
soup = BeautifulSoup(html,'html.parser')

In [23]:
soup.find('meta', {'property': 'og:description'}).get('content')

'라마다 앙코르 바이윈덤 부산해운대'

In [42]:
soup.find('div',class_='blind').text

'\n\t\t\t\t라마다 앙코르 바이윈덤 부산해운대는 세계 최다 호텔을 보유한 윈덤(Wyndham) 그룹의 글로벌 브랜드 호텔로 해운대역(부산 2호선)에서 바로 앞에 위치해 해운대의 해변과 도시를 모두 찾는 고객에게 가장 편안한 여행의 경험을 선사한다.\n해운대 해수욕장까지 도보 500m로 아름다운 바다를 가깝게 만날 수 있으며 부산의 맛을 느낄 수 있는 다양한 먹거리도 쉽게 찾을 수 있다. 호텔에서 10분 거리에 위치한 벡스코는 대중교통으로도 이동할 수 있어 비즈니스로 해운대를 방문하시는 고객에게 교통의 편의까지 더하여 최고의 입지를 자랑한다.\n\t\t\t'

In [35]:
soup.find('meta', {'name': 'description'}).get('content')

'라마다 앙코르 바이윈덤 부산해운대는 세계 최다 호텔을 보유한 윈덤(Wyndham) 그룹의 글로벌 브랜드 호텔로 해운대역(부산 2호선)에서 바로 앞에 위치해 해운대의 해변과 도시를 모두 찾는 고객에게 가장 편안한 여행의 경험을 선사한다.\n해운대 해수욕장까지 도보 500m로 아름다운 바다를 가깝게 만날 수 있으며 부산의 맛을 느낄 수 있는 다양한 먹거리도 쉽게 부산숙소|숙박|해운대구'